In [1]:
import math

In [2]:
class subchain:
    
    # Subchain class 
    # phin: # of VNFs
    # phip: the amount of packets to process
    # ovs_type: kernel or DPDK
    # placement_function: g or d
    # packet_size: 100 or 1500
    
    def __init__(self, phin = 25, phip = 10000, ovs_type = 'kernel', placement_function = 'g', packet_size = 100):
        self.phin = phin
        self.phip = phip
        self.ovs_type = ovs_type
        self.placement_function = placement_function
        self.packet_size = packet_size
        self.get_constant()
        self.f = self.F()
        
    def get_constant(self):
        if self.ovs_type == 'kernel' and self.placement_function == 'g' and self.packet_size == 100:
            self.alpha = 0.586
            self.beta = 0.858
            self.gamma = -1.789
        elif self.ovs_type == 'kernel' and self.placement_function == 'g' and self.packet_size == 1500:
            self.alpha = 0.752
            self.beta = 0.979
            self.gamma = -3.856
        elif self.ovs_type == 'kernel' and self.placement_function == 'd' and self.packet_size == 100:
            self.alpha = 0.660
            self.beta = 0.243
            self.gamma = -2.661
        elif self.ovs_type == 'kernel' and self.placement_function == 'd' and self.packet_size == 1500:
            self.alpha = 1.009
            self.beta = 0.268
            self.gamma = -7.176
        elif self.ovs_type == 'DPDK' and self.placement_function == 'g' and self.packet_size == 100:
            self.alpha = 0.370
            self.beta = 0.467
            self.gamma = 1.543
        elif self.ovs_type == 'DPDK' and self.placement_function == 'g' and self.packet_size == 1500:
            self.alpha = 0.478
            self.beta = 0.578
            self.gamma = 0.194
        elif self.ovs_type == 'DPDK' and self.placement_function == 'd' and self.packet_size == 100:
            self.alpha = 0.217
            self.beta = 0.091
            self.gamma = 3.795
        elif self.ovs_type == 'DPDK' and self.placement_function == 'd' and self.packet_size == 1500:
            self.alpha = 0.157
            self.beta = 0.109
            self.gamma = 4.718
            
    def F(self):
        
        # Compute the cost of this subchain
        
        self.logc = self.alpha * math.log(self.phin) + self.beta * math.log(self.phip) + self.gamma
        return self.logc
    
    def concurrent_F(self,r):
        
        # compute the cost of this subchain concurrent r times
        
        self.logc = self.alpha * math.log(self.phin*r) + self.beta * math.log(self.phip) + self.gamma
        return self.logc
    
    def get_delta(self,r):
        
        # compute the switching cost delta
        
        self.concur_f = self.concurrent_F(r)
        self.delta = r * self.f - self.concur_f
        return self.delta

In [3]:
class multiple_subchain:
    
    # class of multiple subchains
    
    def __init__(self, subchain_list):
        self.r = len(subchain_list)
        self.subchain_list = subchain_list
        self.get_f_list()
        self.get_delta_list()
        
    def get_delta_list(self):
        self.delta_list = []
        for i in range(self.r):
            delta = self.subchain_list[i].get_delta(self.r)
            self.delta_list.append(delta)
            
    def get_f_list(self):
        self.f_list = []
        for i in range(self.r):
            f = self.subchain_list[i].f
            self.f_list.append(f)
            
    def get_cost(self):
        self.cost = sum(self.f_list)-(self.r-1)*sum(self.delta_list)/self.r
        return(self.cost)

In [31]:
import numpy as np
# An example of computing the cost of two subchains in a server

a = subchain(phin = 5, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
b = subchain(phin = 6, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
c = subchain(phin = 4, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
d = subchain(phin = 6, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
e = subchain(phin = 4, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
f = subchain(phin = 6, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
g = subchain(phin = 4, phip = 10000, ovs_type = 'kernel', placement_function = 'd', packet_size = 1500)
x = multiple_subchain([a,b,c, d, e, f, g])
print np.log(x.get_cost())
y = multiple_subchain([a,b,c, d])
print np.log(y.get_cost())
z = multiple_subchain([a])
print np.log(z.get_cost())

4.622223323389673
2.967146379471757
nan


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()


In [ ]:
#def gathering_placement(server_list, )